In [1]:
from data import Data
from model import main

In [2]:
order_data_path = "input_data/order_data.csv"
driver_data_path = "input_data/driver_data.csv"

In [3]:
data_class = Data(
    order_data_path=order_data_path,
    driver_data_path=driver_data_path,
    cost_per_meter_dict={"BIKE": 1, "CAR": 3},
    speed_kmph_dict={"BIKE": 15, "CAR": 30},
    capacity_units_dict={"BIKE": 10, "CAR": 30},
    dummy_depot_count=1,
    dummy_depot_time_windows=[[0, 7200]],
)

Input data sanity check done successfully.


In [4]:
manager, routing, solution = main(data_class)

Routing model created successfully!
Cost of each arc defined successfully!
Pickup & Delivery constraints applied successfully!
Capacity constraints applied successfully!
Time Window constraints applied successfully!
Solution found in 24.801329135894775 seconds !!


In [5]:
print(f"Objective Cost: ${solution.ObjectiveValue()}")
time_dimension = routing.GetDimensionOrDie("Time")
distance_dimension = routing.GetDimensionOrDie("Distance")
total_time = 0
total_distance = 0
max_route_cost = 0
total_route_cost = 0
total_delay = 0
for vehicle_id in range(data_class.num_vehicles):
    index = routing.Start(vehicle_id)
    plan_output = f"Route for vehicle {vehicle_id}:\n"
    route_cost = 0
    route_distance = 0
    route_penalty_cost = 0
    route_delay = 0
    while not routing.IsEnd(index):
        time_var = time_dimension.CumulVar(index)
        dist_var = distance_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
            " -> "
        )
        if solution.Min(time_var) >= 2400:
            route_penalty_cost += (solution.Min(time_var) - 2400) * 1
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += solution.Value(dist_var)
        route_cost += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

    time_var = time_dimension.CumulVar(index)
    dist_var = distance_dimension.CumulVar(index)

    plan_output += (
        f"{manager.IndexToNode(index)}"
        f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
    )

    route_delay += route_penalty_cost
    route_cost += route_penalty_cost * data_class.tw_violation_penalty
    plan_output += f"Time of the route: {solution.Min(time_var)} seconds\n"
    plan_output += f"Cost of the route: ${route_cost}\n"
    plan_output += f"Distance of the route: {solution.Min(dist_var)}\n"
    plan_output += f"Delay of the route: {route_delay}"
    print(plan_output)
    max_route_cost = max(route_cost, max_route_cost)
    total_route_cost += route_cost
    total_time += solution.Min(time_var)
    total_distance += route_distance
    total_delay += route_delay
print(f"Maximum of the route costs: ${max_route_cost}")
print(f"Total route cost: ${total_route_cost}")
print(f"Total route time: {total_time} seconds")
print(f"Total route distance: {total_distance} meters")
print(f"Total route delay: {total_delay} seconds")

Objective Cost: $636478
Route for vehicle 0:
201 Time(0,0) -> 89 Time(1080,1080) -> 27 Time(1308,1308) -> 127 Time(2146,2146) -> 17 Time(2368,2368) -> 189 Time(2405,2405) -> 117 Time(3513,3513) -> 0 Time(3513,3513)
Time of the route: 3513 seconds
Cost of the route: $11810
Distance of the route: 10692
Delay of the route: 1118
Route for vehicle 1:
202 Time(0,0) -> 14 Time(360,360) -> 21 Time(827,827) -> 80 Time(923,923) -> 121 Time(1287,1287) -> 114 Time(1815,1815) -> 180 Time(2316,2316) -> 0 Time(2316,2316)
Time of the route: 2316 seconds
Cost of the route: $8381
Distance of the route: 8381
Delay of the route: 0
Route for vehicle 2:
203 Time(0,0) -> 67 Time(420,1074) -> 59 Time(1200,1200) -> 167 Time(2079,2079) -> 47 Time(2316,2316) -> 159 Time(3020,3020) -> 147 Time(3585,3585) -> 0 Time(3585,3585)
Time of the route: 3585 seconds
Cost of the route: $13247
Distance of the route: 11442
Delay of the route: 1805
Route for vehicle 3:
204 Time(0,0) -> 43 Time(420,420) -> 46 Time(833,833) -> 1